In [1]:
import json
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.preprocessing import StandardScaler
import os
import pickle
from tqdm import tqdm
import lightning as L
import torch.nn as nn
from torch.utils.data import random_split
from lightning.pytorch.callbacks import ModelCheckpoint
import sys
import random

sys.path.append("/home/maxihuber/eeg-foundation/")
L.seed_everything(42)

[rank: 0] Seed set to 42


42

In [26]:
json_path = '/itet-stor/maxihuber/deepeye_storage/foundation_tasks/erp/errp_all.json'

def load_index0(data_index_path):
    with open(data_index_path, 'r') as f:
        train_test_dict = json.load(f)
    train_samples = train_test_dict['train']
    test_samples = train_test_dict['test']
    return train_samples, test_samples

def load_index1(data_index_paths):
    all_samples = []
    for data_index_path in data_index_paths:
        with open(data_index_path, 'r') as f:
            subset_dict = json.load(f)
        all_samples.append(list(subset_dict.values())[0])
    return all_samples[0], all_samples[1], all_samples[2]

train_index, test_index = load_index0(json_path)
outputs = set([sample["label"] for sample in train_index] + [sample["label"] for sample in test_index])
print(outputs)
print(len(outputs))

{'Target is located in the right', 'without error-related potential', 'The cursor moves to the left', 'The feedback consisted in the selected item is presented on the screen', 'The cursor moves to the right', 'with error-related potential', 'Target is located in the left'}
7


In [ ]:
with open('/home/maxihuber/eeg-foundation/src/data/components/channels_to_id.json', 'r') as f:
    pkl_channels = set(json.load(f).keys())
    cli_channels = set(['AF3', 'AF4', 'AF7', 'AF8', 'AFz', 
                     'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 
                     'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CPz', 'Cz', 
                     'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 
                     'FC1', 'FC2', 'FC3', 'FC4', 'FC5', 'FC6', 'FCz', 
                     'FT7', 'FT8', 'Fp1', 'Fp2', 'Fz', 'Mastoids', 
                     'O1', 'O2', 'Oz', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 
                     'PO3', 'PO4', 'PO7', 'PO8', 'POz', 'Pz', 'T7', 'T8', 'TP7', 'TP8', 
                     'Veog', 'X', 'Y', 'Z'])
    task_channels = pkl_channels | cli_channels